In [17]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

from pathlib import Path
import h5py
import gemmi
from ess.nmx.mcstas import McStasWorkflow
import numpy as np

from ess.nmx.types import *
from ess.nmx.reduction import NMXData, NMXReducedData, merge_panels
from ess.nmx.nexus import export_as_nexus

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
from nmx_workflow.process import BinnedMcStasData

In [86]:
import tracemalloc
tracemalloc.start()
binned_McStasdata = BinnedMcStasData(input_file="/Users/aaronfinke/nmx_workflow/nmx_workflow/data/raw/main/mccode_phi0.h5",
                                nbins=50, 
                                probability_weight=1e5)
binned_McStasdata.run()
binned_data = binned_McStasdata.get_binned_data()
print(tracemalloc.get_traced_memory())
tracemalloc.stop()

2025-02-22 17:43:19.548 | INFO     | nmx_workflow.process:min_max_probability_tof:111 - minmax_t frame 0
2025-02-22 17:43:22.121 | INFO     | nmx_workflow.process:min_max_probability_tof:111 - minmax_t frame 1
2025-02-22 17:43:24.463 | INFO     | nmx_workflow.process:min_max_probability_tof:111 - minmax_t frame 2
2025-02-22 17:43:26.857 | INFO     | nmx_workflow.process:min_max_probability_tof:124 - max_t: 0.1429755220620797, min_t: 0.07228479758975796
2025-02-22 17:43:26.857 | INFO     | nmx_workflow.process:min_max_probability_tof:125 - max_p: 1.2608385526453254e-05
2025-02-22 17:43:26.939 | INFO     | nmx_workflow.process:run:240 - Storage size of datasets: 17.05 GB
2025-02-22 17:43:26.944 | INFO     | nmx_workflow.process:do_tof_binning:185 - binning frame 0
2025-02-22 17:43:35.048 | INFO     | nmx_workflow.process:do_tof_binning:196 - time to process frame 0: 8.10 s
2025-02-22 17:43:35.120 | INFO     | nmx_workflow.process:do_tof_binning:185 - binning frame 1
2025-02-22 17:43:40.9

In [62]:
from nmx_workflow.config import PROCESSED_DATA_DIR
with open(PROCESSED_DATA_DIR/'binned.npy','wb') as fp:
    np.save(fp,binned_data)

In [89]:
import nmx_workflow.refl_loader as rl
refl_file = Path("/Users/aaronfinke/nmx_workflow/nmx_workflow/data/processed/dials/integrated.refl")
refls = rl.load(refl_file)

<class 'pathlib.PosixPath'>


In [94]:
for k in refls.keys():
    print(k)

L1
background.dispersion
background.mean
background.mse
background.sum.value
background.sum.variance
bbox
d
entering
flags
id
imageset_id
intensity.sum.value
intensity.sum.variance
miller_index
num_pixels.background
num_pixels.background_used
num_pixels.foreground
num_pixels.valid
panel
partiality
rlp
s0
s0_cal
s1
tof_cal
wavelength_cal
xyzcal.mm
xyzcal.px
xyzobs.px.value
experiment_identifier


In [129]:
import pandas as pd
df = pd.DataFrame.from_dict({
    'H':refls['miller_index'][:,0],
    'K':refls['miller_index'][:,1],
    'L':refls['miller_index'][:,2],
    'd':refls['d'],
    'LAMBDA':refls['wavelength_cal'],
    'I':refls['intensity.sum.value'],
    'varI':refls['intensity.sum.variance'],
    'imageset_id':refls['imageset_id']

})
df

,H,K,L,d,LAMBDA,I,varI,imageset_id
0,-7,-2,12,2.933515,3.468253,436400.25000,7.339634e+05,0
1,-7,2,12,2.933515,3.468440,373579.65625,6.805508e+05,0
2,-6,-3,11,3.198319,3.533888,822237.81250,1.028331e+06,0
3,-9,-3,12,2.612049,3.534606,272716.71875,4.172071e+05,0
4,-7,-1,12,2.963473,3.539501,488630.90625,6.823005e+05,0
...,...,...,...,...,...,...,...,...
2552,1,2,-18,2.445542,3.478563,650900.87500,7.914811e+05,5
2553,2,-2,-16,2.717414,3.514341,455611.28125,5.589029e+05,5
2554,1,-1,-18,2.462818,3.528017,276295.03125,3.489453e+05,5
2555,1,0,-18,2.468659,3.544726,328746.84375,4.104788e+05,5


In [132]:
df.sort_values(by=['H','K','L'], key=abs, inplace=True)

In [188]:
mult_dict = df.groupby(['H', 'K', 'L']).size().to_dict()
nref = len(df)
ds = [tuple(x) for x in df[['H','K','L']].values]
mult = [mult_dict[ds[i]] for i in range(nref)]

In [190]:
df.insert(len(df.columns), "MULT", np.array(mult).astype("int32"))
df

,H,K,L,d,LAMBDA,I,varI,imageset_id,MULT
1718,0,0,-14,3.182019,3.177821,6.357652e+05,7.811506e+05,3,1
1705,0,0,-16,2.784267,2.780593,1.043345e+06,1.241363e+06,3,1
812,0,-1,-4,10.634998,3.511426,1.524728e+06,1.630176e+06,1,1
813,0,1,-4,10.634998,3.512081,1.527934e+06,1.631438e+06,1,1
772,0,-1,-5,8.646263,2.901240,4.189829e+06,4.396597e+06,1,1
...,...,...,...,...,...,...,...,...,...
966,-15,1,6,2.208555,3.507222,3.527870e+05,6.432466e+05,2,1
1704,-15,-1,-35,1.115150,1.905312,9.109375e+00,4.077845e+04,3,1
967,-17,-2,3,2.013414,3.539509,4.106455e+05,5.865279e+05,2,1
972,-17,2,3,2.013414,3.539514,3.390641e+05,5.078285e+05,2,1


In [191]:
df[(df['H']==-1) & (df['K']==-6) & (df['L']==-1)]

,H,K,L,d,LAMBDA,I,varI,imageset_id,MULT
20,-1,-6,-1,5.833896,1.955323,2.456019e+05,2.718069e+05,0,6
452,-1,-6,-1,5.833896,2.191706,8.425216e+05,9.123292e+05,1,6
863,-1,-6,-1,5.833896,2.361494,8.444862e+05,9.121949e+05,2,6
1304,-1,-6,-1,5.833896,2.459530,9.475338e+05,1.016300e+06,3,6
1729,-1,-6,-1,5.833896,2.482834,1.173158e+06,1.253782e+06,4,6
2144,-1,-6,-1,5.833896,2.430698,3.229425e+05,3.478225e+05,5,6


In [195]:
df.insert(df.columns.get_loc('varI'),"SIGI",np.sqrt(df['varI']))
df

,H,K,L,d,LAMBDA,I,SIGI,varI,imageset_id,MULT
1718,0,0,-14,3.182019,3.177821,6.357652e+05,883.827224,7.811506e+05,3,1
1705,0,0,-16,2.784267,2.780593,1.043345e+06,1114.164766,1.241363e+06,3,1
812,0,-1,-4,10.634998,3.511426,1.524728e+06,1276.783654,1.630176e+06,1,1
813,0,1,-4,10.634998,3.512081,1.527934e+06,1277.277623,1.631438e+06,1,1
772,0,-1,-5,8.646263,2.901240,4.189829e+06,2096.806381,4.396597e+06,1,1
...,...,...,...,...,...,...,...,...,...,...
966,-15,1,6,2.208555,3.507222,3.527870e+05,802.026574,6.432466e+05,2,1
1704,-15,-1,-35,1.115150,1.905312,9.109375e+00,201.936736,4.077845e+04,3,1
967,-17,-2,3,2.013414,3.539509,4.106455e+05,765.851120,5.865279e+05,2,1
972,-17,2,3,2.013414,3.539514,3.390641e+05,712.620867,5.078285e+05,2,1
